In [2]:
import pandas as pd 
import numpy as np
df = pd.read_excel('orders-2022-01-16-14-02-13.xlsx')
dfstart = pd.read_excel('orders-2022-01-16-14-02-13.xlsx')
df.head()

,Order Number,Source,Ship the first kit to my billing address,User Email,Subscription Start Date,renewal order,Customer Total Orders,First Kit,Order Status,Order Date,...,Subscriber email,Subscription Last order date,Subscription On hold date,Subscription Cancelleted date,Subscription End date,Subscription Coupon,Subscription Total refund,Subscription Total revenue,Start Month,End Month
0,133615,google,NaN,middletonak@gmail.com,2021-12-01 04:32:34,130974.0,5,NaN,Processing,2021-12-31 20:32:34,...,middletonak@gmail.com,2021-12-31 20:32:34,NaN,NaN,NaN,"black,",0.0,29.99,12-2021,NaN
1,133614,google,NaN,middletonak@gmail.com,2021-12-01 04:28:07,130972.0,5,NaN,Processing,2021-12-31 20:28:38,...,middletonak@gmail.com,2021-12-31 20:28:38,NaN,NaN,NaN,"black,",0.0,59.93,12-2021,NaN
2,133613,fb,NaN,kathleenmiller701@gmail.com,2021-12-01 04:26:16,130970.0,2,NaN,Failed,2021-12-31 20:26:39,...,kathleenmiller701@gmail.com,2021-12-31 20:26:39,2021-12-31 20:26:39,NaN,NaN,NaN,0.0,39.95,12-2021,NaN
3,133612,(direct),NaN,ross.826@osu.edu,2021-12-01 03:41:16,130968.0,4,NaN,Processing,2021-12-31 19:42:30,...,ross.826@osu.edu,2021-12-31 19:42:30,NaN,2022-01-01 05:00:23,2022-02-01 03:41:16,"black,",0.0,32.06,12-2021,02-2022
4,133611,(direct),NaN,ross.826@osu.edu,2021-12-01 03:40:04,130966.0,4,NaN,Processing,2021-12-31 19:40:30,...,ross.826@osu.edu,2021-12-31 19:40:30,NaN,NaN,NaN,"black,",0.0,32.09,12-2021,NaN


In [3]:
def update_sub_start_date(df):
    print('Добавляем значения в поле Subscription Start Date для Bundle и Gift.')
    a = list(df['Subscription Start Date'])
    b = list(df['Order Date'])

    for i in range(len(a)):
        if str(a[i]) == 'nan':
            a[i] = str(b[i])
    df['Subscription Start Date'] = a
    return df

def update_full_names_for_catefories(df):
    print('Переименовываем названия наборов.')
    a = list(df['Full names for categories'])
    for i in range(len(a)):
        try:
            rez = ''
            if '3+' in a[i]:
                rez += '3+'
            if ('5+' in a[i]) or ('bundle' in a[i]) or ('Bundle' in a[i]):
                rez += '5+'
            if ('C&' in a[i]) or ('8+' in a[i]):
                rez += '8+'
            if len(rez) > 0:
                a[i] = rez
        except:
            continue
    df['Full names for categories'] = a
    return df

def del_test_coupons(df):
    print('Удаляем тестовые купоны.')
    coupons = list(df['Coupon Code'])
    for i in range(len(coupons)):
        try:
            if 'test' in coupons[i] or 'altdev' in coupons[i] or 'Altdev' in coupons[i]:
                coupons[i] = 'test'
        except:
            continue
    df['Coupon Code'] = coupons
    df.drop(df[df['Coupon Code'].isin(['test'])].index,inplace = True) 
    
    coupons = list(df['Subscription Coupon'])
    for i in range(len(coupons)):
        try:
            if 'test' in coupons[i] or 'altdev' in coupons[i] or 'Altdev' in coupons[i]:
                coupons[i] = 'test'
        except:
            continue
    df['Subscription Coupon'] = coupons
    df.drop(df[df['Subscription Coupon'].isin(['test'])].index,inplace = True) 
    return df             

def del_wrong_mails(df):
    print('Удалем некорректные почты.')
    d = ['F0e0n0i0k0s@gmail.com', 'masha20795@gmail.com','test@test.by',
         'mkrasnoschekova@gmail.com','anton.rogozhkin@gmail.com','plokhotnov@gmail.com','Ksenia_levenkova@yahoo.com']
    mails = list(df['Email (Billing)'])
    for i in range(len(mails)):
        if '@oysterkit.com' in mails[i] or '.ru' in mails[i]:
            mails[i] = 'test'
    df['Email (Billing)'] = mails
    df.drop(df[df['Email (Billing)'].isin(['test'])].index,inplace = True)
    return df

def add_sub_month(df):
    print('Добавляем столбцы sub_month к таблице.')
    a = list(df['Subscription Start Date'])
    for i in range(len(a)):
        try:
            a[i] = a[i].split()[0][:-3]
        except:
            a[i] = 'NaN'      
    df['sub_month'] = a
    return df

def rename_Item_name(df,file):
    print('Приводим к одному названию поле Item Name.')
    f = open(file)
    d = {}
    line = f.readline()
    while line:
        x = line.split('//')
        d[x[0].strip()] = x[1].strip()
        line = f.readline()
    names = list(df['Item Name'])
    for i in range(len(names)):
        try:
            names[i] = d[names[i].strip()]
        except:
            names[i] = 'test'
    df['Item Name'] = names
    return df

def add_year_and_month(df):
    print('Добавляем столбец Year и Month.')
    a = df['Order Date']
    year = []
    month = []
    for x in a:
        r = str(x).split()[0].split('-')
        year.append(r[0])
        month.append(int(r[1]))
    df['Year'] = year
    df['Month'] = month
    
    return df

def del_rus_names(df):
    print('Удаляем русифицированные имена и фамилии.')
    l = []
    for x in range(1040,1104):
        l.append(chr(x))

    fname = list(df['First Name (Billing)'])
    lname = list(df['Last Name (Billing)'])

    for i in range(len(fname)):
        try:
            for x in l:
                if x in fname[i]:
                    fname[i] = 'test'
        except:
            continue
    for i in range(len(lname)):
        try:
            for x in l:
                if x in lname[i]:
                    lname[i] = 'test'
        except:
            continue
    df['First Name (Billing)'] = fname
    df['Last Name (Billing)'] = lname

    df.drop(df[df['First Name (Billing)'].isin(['test'])].index,inplace = True)
    df.drop(df[df['Last Name (Billing)'].isin(['test'])].index,inplace = True)
    return df

def del_test_orders(df):
    print('Удаляем тестовые заказы.')
    bad_names = ['test','AltestTest','test 1','test 2','Test',
                 'alttest','Alttest','Altestat','test2','Test1','Test2','Ulugbek','ulugbek','Zayniev','zayniev']
    df.drop(df[df['Item Name'].isin(['test'])].index,inplace = True)
    df.drop(df[df['Last Name (Billing)'].isin(bad_names)].index,inplace = True)
    df.drop(df[df['First Name (Billing)'].isin(bad_names)].index,inplace = True)
    df.drop(df[df['Order Status'].isin(['Failed'])].index,inplace = True)
    return df        

In [4]:
df = update_sub_start_date(df)
df = del_wrong_mails(df)
df = del_test_coupons(df)
df = rename_Item_name(df,'lines.txt')
df = update_full_names_for_catefories(df)
df = del_test_orders(df)
df = del_rus_names(df)
df = add_year_and_month(df)
df = add_sub_month(df)

Добавляем значения в поле Subscription Start Date для Bundle и Gift.
Удалем некорректные почты.
Удаляем тестовые купоны.
Приводим к одному названию поле Item Name.
Переименовываем названия наборов.
Удаляем тестовые заказы.
Удаляем русифицированные имена и фамилии.
Добавляем столбец Year и Month.
Добавляем столбцы sub_month к таблице.


In [5]:
def update_to_pivot(data):
    func = {'Order Total Amount': np.sum, 'Order Refund Amount': np.sum, 'Quantity': len}
    pivot = pd.pivot_table(data, index = ['sub_month','Item Name'], columns = ['Month'], 
                                  values = ['Order Total Amount','Order Refund Amount','Quantity',], aggfunc = func)
    pivot = pivot.stack().stack().unstack('Month')
    pivot.to_excel('output.xlsx')  
    pivot = pd.read_excel('output.xlsx')
    return pivot

def add_local_stats(pivot):
    d_Churn = {list(pivot.columns)[2]: 'Churn'}

    count = 0        
    for i in range(0, len(pivot) - 2, 3):
        line = list(pivot.iloc[i + 2])[3:]
        for j in range(len(line)):
            if str(line[j]) == 'nan':
                line[j] = 0
        ch = [0, 0]
        for j in range(1, len(line)):
            if line[j - 1] == 0:
                ch.append(0)
            else:
                ch.append(line[j] - line[j - 1])
        for j in range(1, 13):
            d_Churn[j] = ch[j]
        new_line  = pd.DataFrame(data = [d_Churn],
                           columns = list(pivot.columns))
        if i == 0:
            res_data = pd.concat([pivot.iloc[i:i + 3], new_line], axis = 0, names = list(pivot.columns))
        else:
            res_data = pd.concat([res_data,pivot.iloc[i:i + 3]], axis = 0, names = list(pivot.columns))
            res_data = pd.concat([res_data, new_line], axis = 0, names = list(pivot.columns))
    return res_data
    
def add_total_stats(pivot):
    hih = list(pivot['sub_month'])
    indices = []
    for i in range(len(hih)):
        try:
            float(hih[i])
        except:
            indices.append(i)
            continue
    indices.append(len(hih))
    count = 0
    for ind in range(len(indices) - 1):
        i = indices[ind]
        j = indices[ind + 1]
        update_block = pivot.iloc[i:j]
        #add Quantity and Revenue
        d_Quantity = {list(pivot.columns)[0]: 'Total Quantity', list(pivot.columns)[1]: '---', list(pivot.columns)[2]: '---'}
        d_Revenue = {list(pivot.columns)[0]: 'Total Amount', list(pivot.columns)[1]: '---', list(pivot.columns)[2]: '---'}
        d_Refund = {list(pivot.columns)[0]: 'Total Refund', list(pivot.columns)[1]: '---', list(pivot.columns)[2]: '---'}
        d_Churn = {list(pivot.columns)[0]: 'Total Churn', list(pivot.columns)[1]: '---', list(pivot.columns)[2]: '---'}
        d_empty = {}
        for k in range(1, 13):
            d_Quantity[k] = 0
            d_Revenue[k] = 0
            d_Churn[k] = 0
            d_Refund[k] = 0
        for k in range(i + 2, j, 4):
            q = list(pivot.iloc[k])[3:]
            amount = list(pivot.iloc[k - 1])[3:]
            refund = list(pivot.iloc[k - 2])[3:]
            ch =  list(pivot.iloc[k + 1])[3:]
            for l in range(len(q)):
                if str(q[l]) == 'nan':
                    continue
                d_Quantity[l + 1] += q[l]
            for l in range(len(ch)):
                d_Churn[l + 1] += ch[l]
            for l in range(len(amount)):
                if str(amount[l]) == 'nan':
                    continue
                d_Revenue[l + 1] += amount[l]
            for l in range(len(refund)):
                if str(refund[l]) == 'nan':
                    continue
                d_Refund[l + 1] += refund[l]
                
               
        new_line  = pd.DataFrame(data = [d_Refund],
                       columns = list(pivot.columns))
        new_line2 = pd.DataFrame(data = [d_Revenue],
                       columns = list(pivot.columns))
        new_line3 = pd.DataFrame(data = [d_Quantity],
                       columns = list(pivot.columns))
        new_line4 = pd.DataFrame(data = [d_Churn],
                       columns = list(pivot.columns))
        new_line5 = pd.DataFrame(data = [d_empty],
                       columns = list(pivot.columns))
        if i == 0:
            res_data = pd.concat([pivot.iloc[i:j], new_line], axis = 0, names = list(pivot.columns))
            res_data = pd.concat([res_data, new_line2], axis = 0, names = list(pivot.columns))
            res_data = pd.concat([res_data, new_line3], axis = 0, names = list(pivot.columns))
            res_data = pd.concat([res_data, new_line4], axis = 0, names = list(pivot.columns))
            res_data = pd.concat([res_data, new_line5], axis = 0, names = list(pivot.columns))
            
        else:
            res_data = pd.concat([res_data,pivot.iloc[i:j]], axis = 0, names = list(pivot.columns))
            res_data = pd.concat([res_data, new_line], axis = 0, names = list(pivot.columns))
            res_data = pd.concat([res_data, new_line2], axis = 0, names = list(pivot.columns))
            res_data = pd.concat([res_data, new_line3], axis = 0, names = list(pivot.columns))
            res_data = pd.concat([res_data, new_line4], axis = 0, names = list(pivot.columns))
            res_data = pd.concat([res_data, new_line5], axis = 0, names = list(pivot.columns))
    return res_data 

In [6]:
df3 = df[df['Full names for categories'].isin(['3+','3+5+','3+5+8+'])]
df5 = df[df['Full names for categories'].isin(['5+','3+5+','3+5+8+'])]
df8 = df[df['Full names for categories'].isin(['8+','3+5+','3+5+8+'])]

rename_d = {'Unnamed: 2': 'Months ->', 1: 'jan', 2: 'feb', 3: 'mar', 4: 'apr', 5: 'may', 6: 'june',
           7: 'july', 8: 'aug', 9: 'sep', 10: 'oct', 11: 'nov', 12: 'dec'}
pivot3 = update_to_pivot(df3)
pivot3 = add_local_stats(pivot3)
pivot3 = add_total_stats(pivot3)
pivot3 = pivot3.set_index('sub_month')
pivot3.rename(columns = rename_d, inplace = True)

pivot5 = update_to_pivot(df5)
pivot5 = add_local_stats(pivot5)
pivot5 = add_total_stats(pivot5)
pivot5 = pivot5.set_index('sub_month')
pivot5.rename(columns = rename_d, inplace = True)

pivot8 = update_to_pivot(df8)
pivot8 = add_local_stats(pivot8)
pivot8 = add_total_stats(pivot8)
pivot8 = pivot8.set_index('sub_month')
pivot8.rename(columns = rename_d, inplace = True)

with pd.ExcelWriter('cohort analysis 2021.xlsx') as writer:  
    dfstart.to_excel(writer, sheet_name = 'сырая выгрузка')
    df.to_excel(writer, sheet_name = 'обработанная таблица')
    pivot3.to_excel(writer, sheet_name = 'cohorts 3+')
    pivot5.to_excel(writer, sheet_name = 'cohorts 5+')
    pivot8.to_excel(writer, sheet_name = 'cohorts 8+')

#старая выгрузка
df3 = df[df['Full names for categories'].isin(['3+','3+5+','3+5+8+'])]
df5 = df[df['Full names for categories'].isin(['5+','3+5+','3+5+8+'])]
df8 = df[df['Full names for categories'].isin(['8+','3+5+','3+5+8+'])]

pivot_quantity3 = pd.pivot_table(df3,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Quantity'],aggfunc=len)
pivot_order_total_amount3 = pd.pivot_table(df3,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Order Total Amount'],aggfunc = np.sum)
pivot_order_refund_amount3 = pd.pivot_table(df3,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Order Refund Amount'],aggfunc = np.sum)

pivot_quantity5 = pd.pivot_table(df5,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Quantity'],aggfunc=len)
pivot_order_total_amount5 = pd.pivot_table(df5,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Order Total Amount'],aggfunc = np.sum)
pivot_order_refund_amount5 = pd.pivot_table(df5,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Order Refund Amount'],aggfunc = np.sum)

pivot_quantity8 = pd.pivot_table(df8,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Quantity'],aggfunc=len)
pivot_order_total_amount8 = pd.pivot_table(df8,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Order Total Amount'],aggfunc = np.sum)
pivot_order_refund_amount8 = pd.pivot_table(df8,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Order Refund Amount'],aggfunc = np.sum)

with pd.ExcelWriter('cohort analysis new v2.xlsx') as writer:  
    dfstart.to_excel(writer, sheet_name = 'сырая выгрузка (2021)')
    df.to_excel(writer, sheet_name = 'обработанная таблица')
    
    pivot_quantity3.to_excel(writer, sheet_name = 'quantity 3+')
    pivot_order_total_amount3.to_excel(writer, sheet_name = 'total_amount 3+')
    pivot_order_refund_amount3.to_excel(writer, sheet_name = 'total_refund 3+')
    
    pivot_quantity5.to_excel(writer, sheet_name = 'quantity 5+')
    pivot_order_total_amount5.to_excel(writer, sheet_name = 'total_amount 5+')
    pivot_order_refund_amount5.to_excel(writer, sheet_name = 'total_refund 5+')
    
    pivot_quantity8.to_excel(writer, sheet_name = 'quantity 8+')
    pivot_order_total_amount8.to_excel(writer, sheet_name = 'total_amount 8+')
    pivot_order_refund_amount8.to_excel(writer, sheet_name = 'total_refund 8+')